# Preamble

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# installs
!pip install gtfs-kit

     |████████████████████████████████| 62 kB 702 kB/s 
     |████████████████████████████████| 10.0 MB 6.6 MB/s 
     |████████████████████████████████| 1.0 MB 47.5 MB/s 
     |████████████████████████████████| 994 kB 49.0 MB/s 
     |████████████████████████████████| 15.4 MB 39 kB/s 
     |████████████████████████████████| 6.3 MB 22.9 MB/s 
  Created wheel for json2html: filename=json2html-1.3.0-py3-none-any.whl size=7609 sha256=a96e7ef6c10c3a1a35953a06ca0d26df9ce5e988f4093ea5762996ded9400351
  Stored in directory: /root/.cache/pip/wheels/e5/99/37/e1839a5ad733e0d6abb7e0419fd913e8926ddf96408239ce01
  Created wheel for pycountry: filename=pycountry-19.8.18-py2.py3-none-any.whl size=10627377 sha256=56aa5a8e31af447888b8ca909e5f9f2650ab18960a45745eaec9c728c1e839c0
  Stored in directory: /root/.cache/pip/wheels/00/77/65/5f318c3c30b046b450a385ecfc690125f8a270cdb0f3f483b4
  Created wheel for utm: filename=utm-0.7.0-py3-none-any.whl size=6109 sha256=4c491e35815dd46b82399b0bf43f541659525b4b4e5

In [ ]:
# imports
import json
import random
import requests
import datetime
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

# Inputs

In [ ]:
# constant
key = 'cb50405b-9cce-440c-967c-09945d849519'

# variable
route_str = 'M15'

In [ ]:
read_segments = f'/content/drive/My Drive/Bus Watcher Spec Project/Projects/Alex Amy + Sanket Shah/Data/Bus/Segment Data - Raw/{route_str}_2021-10-18.csv'
save_segments = f'/content/drive/My Drive/Bus Watcher Spec Project/Projects/Alex Amy + Sanket Shah/Data/Bus/Segment Data - Processed/{route_str}_2021-10-18.csv'
save_stops = f'/content/drive/My Drive/Bus Watcher Spec Project/Projects/Alex Amy + Sanket Shah/Data/Bus/Route Data/{route_str}_stops.json'

# Read

In [ ]:
route = pd.read_csv(read_segments)

In [ ]:
route['timestamp'] = pd.to_datetime(route['timestamp'])
route['next_stop_eta'] = pd.to_datetime(route['next_stop_eta'])

In [ ]:
# sort timestamp
route = route.sort_values('timestamp')

# Clean

In [ ]:
route['progress_rate'].value_counts()

normalProgress    85086
noProgress         3913
unknown            1626
Name: progress_rate, dtype: int64

In [ ]:
route['progress_rate'].value_counts(normalize=True)

normalProgress    0.938880
noProgress        0.043178
unknown           0.017942
Name: progress_rate, dtype: float64

In [ ]:
# only consider vehicles reporting normal progress (i.e. moving, generally). source: https://bustime.mta.info/wiki/Developers/SIRIMonitoredVehicleJourney
route = route[route['progress_rate'] == 'normalProgress']

In [ ]:
route['next_stop_id'].isna().sum()

66

In [ ]:
route['next_stop_id'].isna().sum() / route.shape[0]

0.0007756857767435301

In [ ]:
route = route[route['next_stop_id'].isna() == False]

In [ ]:
route['passenger_count'].isna().sum()

5527

In [ ]:
route['passenger_count'].isna().sum() / route.shape[0]

0.06500823335685721

In [ ]:
# compute max number of consecutive NaN values in passenger_count
max_consecutive_passenger_count_NaNs = {}
unique_trip_ids = set(route['unique_trip_id'])

for uuid in unique_trip_ids:
  df = route[route['unique_trip_id'] == uuid]
  passenger_counts = df['passenger_count']
  NaN_groups = passenger_counts.notna().cumsum()[passenger_counts.isna()]
  lengths_consecutive_NaN = NaN_groups.groupby(NaN_groups).agg(len)
  longest_NaN_group = lengths_consecutive_NaN.max()
  if np.isnan(longest_NaN_group):
    longest_NaN_group = 0
  max_consecutive_passenger_count_NaNs[uuid] = longest_NaN_group

In [ ]:
max(max_consecutive_passenger_count_NaNs.values())

58

In [ ]:
delinquent_uuids = [(max_consecutive_passenger_count_NaNs[uuid], uuid) for uuid in max_consecutive_passenger_count_NaNs if max_consecutive_passenger_count_NaNs[uuid] > 10]

In [ ]:
for (longest_NaN_group, uuid) in random.sample(delinquent_uuids, 25):
  df = route[route['unique_trip_id'] == uuid]
  timestamps = list(df['timestamp'])
  start_time = timestamps[0].time()
  end_time = timestamps[-1].time()
  print(f'{start_time} --> {end_time} - {longest_NaN_group} / {df.shape[0]}')

23:02:57 --> 00:03:32 - 12 / 43
10:19:16 --> 11:55:03 - 11 / 54
13:54:54 --> 14:16:30 - 12 / 12
10:41:09 --> 12:08:43 - 45 / 45
13:35:20 --> 14:50:08 - 47 / 47
20:09:11 --> 20:56:01 - 36 / 36
14:00:15 --> 15:34:08 - 11 / 55
15:17:18 --> 16:52:31 - 53 / 53
11:04:36 --> 12:13:24 - 39 / 39
17:04:19 --> 18:11:49 - 38 / 40
12:58:41 --> 14:07:34 - 36 / 42
20:06:20 --> 21:12:39 - 30 / 30
14:45:59 --> 16:13:22 - 41 / 42
13:04:19 --> 15:17:13 - 11 / 56
10:37:36 --> 11:59:26 - 49 / 49
22:00:54 --> 23:17:44 - 52 / 52
12:24:04 --> 14:12:07 - 44 / 44
01:16:26 --> 01:57:13 - 13 / 32
21:56:46 --> 22:54:11 - 25 / 44
21:52:15 --> 22:55:13 - 13 / 13
11:48:58 --> 13:13:05 - 26 / 47
18:29:15 --> 19:25:30 - 41 / 41
17:38:38 --> 19:00:08 - 46 / 46
20:26:55 --> 21:38:35 - 15 / 37
23:04:24 --> 23:22:07 - 14 / 14


In [ ]:
# remove unique_trip_ids with no passenger_count readings
for uuid in unique_trip_ids:
  df = route[route['unique_trip_id'] == uuid]
  num_not_nan =  df['passenger_count'].notna().sum()
  if num_not_nan == 0:
    route = route[route['unique_trip_id'] != uuid].copy()

In [ ]:
# replace remaining NaN passenger_count values with 0
route['passenger_count'] = route['passenger_count'].fillna(0)

In [ ]:
route.shape[0]

83108

# Stops

**TODO: Figure out way to handle route changes (e.g. M15 and Bx12)...**

In [ ]:
service_dates = sorted(list(set(route['service_date'])))

In [ ]:
stops_dict = {}

prior_service_date = service_dates[0]
url = 'http://bustime.mta.info/api/where/stops-for-route/MTA%20NYCT_{route_str}.json?key={key}&includePolylines=false&time={service_date}&version=2'.format(route_str=route_str.upper(), key=key, service_date=prior_service_date)
response = requests.get(url).json()
response = response['data']['entry']['stopGroupings'][0]['stopGroups']
assert len(response) == 2
if response[0]['id'] == '1':
  prev_stops_dir_1 = response[0]['stopIds']
  prev_stops_dir_0 = response[1]['stopIds']
else:
  assert response[0]['id'] == '0'
  prev_stops_dir_1 = response[1]['stopIds']
  prev_stops_dir_0 = response[0]['stopIds']

for service_date in service_dates[1:]:
  url = 'http://bustime.mta.info/api/where/stops-for-route/MTA%20NYCT_{route_str}.json?key={key}&includePolylines=false&time={service_date}&version=2'.format(route_str=route_str.upper(), key=key, service_date=service_date)
  response = requests.get(url).json()
  response = response['data']['entry']['stopGroupings'][0]['stopGroups']
  assert len(response) == 2
  if response[0]['id'] == '1':
    new_stops_dir_1 = response[0]['stopIds']
    new_stops_dir_0 = response[1]['stopIds']
  else:
    assert response[0]['id'] == '0'
    new_stops_dir_1 = response[1]['stopIds']
    new_stops_dir_0 = response[0]['stopIds']
  if (new_stops_dir_1 == prev_stops_dir_1) and (new_stops_dir_0 == prev_stops_dir_0):
    prior_service_date = service_date
    if service_date != service_dates[-1]:
      prev_stops_dir_1 = new_stops_dir_1
      prev_stops_dir_0 = new_stops_dir_0
    else:
      print('Stops match for all dates in range')
      stops_dict[1] = new_stops_dir_1
      stops_dict[0] = new_stops_dir_0
      with open(save_stops, 'w') as f:
        json.dump(stops_dict, f)
  else:
    if new_stops_dir_1 != prev_stops_dir_1:
      print(f'Service change in direction 1 on {service_date}:')
      for prev_stop_dir_1, new_stop_dir_1 in zip(prev_stops_dir_1, new_stops_dir_1):
        print(prev_stop_dir_1, new_stop_dir_1)
      print('\n')
    if new_stops_dir_0 != prev_stops_dir_0:
      print(f'Service change in direction 0 on {service_date}:')
      for prev_stop_dir_0, new_stop_dir_0 in zip(prev_stops_dir_0, new_stops_dir_0):
        print(prev_stop_dir_0, new_stop_dir_0)
      print('\n')
    break

Service change in direction 1 on 2021-08-03:
MTA_401732 MTA_401732
MTA_401738 MTA_401738
MTA_401739 MTA_401739
MTA_401740 MTA_401740
MTA_401741 MTA_401741
MTA_401742 MTA_401742
MTA_803228 MTA_803228
MTA_401744 MTA_401744
MTA_401745 MTA_401745
MTA_401746 MTA_401746
MTA_401747 MTA_401747
MTA_401748 MTA_401748
MTA_401749 MTA_401749
MTA_403342 MTA_403342
MTA_405233 MTA_405233
MTA_405307 MTA_405307
MTA_401753 MTA_401753
MTA_401754 MTA_401754
MTA_401755 MTA_401755
MTA_401756 MTA_401756
MTA_401758 MTA_401758
MTA_401759 MTA_401759
MTA_401760 MTA_401760
MTA_401761 MTA_401761
MTA_401762 MTA_401762
MTA_401763 MTA_401763
MTA_401764 MTA_401764
MTA_401765 MTA_401765
MTA_403887 MTA_403887
MTA_401768 MTA_401768
MTA_401769 MTA_401769
MTA_401771 MTA_401771
MTA_401772 MTA_401772
MTA_404909 MTA_404909
MTA_903266 MTA_903266
MTA_401775 MTA_401775
MTA_405336 MTA_405336
MTA_401778 MTA_401778
MTA_401779 MTA_401779
MTA_401780 MTA_401780
MTA_401781 MTA_401781
MTA_401782 MTA_401782
MTA_404105 MTA_404105
MTA_40178

In [ ]:
# zip direction and next_stop_id to look up position of next_stop_id along appropriate route stop sequence
route['direction-next_stop_id'] = list(zip(route['direction'], route['next_stop_id']))

# look up position of next_stop_id along appropriate route stop sequence and drop direction-next_stop_id column (not needed)
route['next_stop_id_pos'] = route['direction-next_stop_id'].apply(lambda x: stops_dict[x[0]].index(x[1]))
route = route.drop(columns='direction-next_stop_id')

# prepend None to route stop sequence to look up prior_stop_id using next_stop_id position and drop direction-stop_sequence column (not needed)
stops_dict[1].insert(0, None)
stops_dict[0].insert(0, None)
route['direction-stop_sequence'] = list(zip(route['direction'], route['next_stop_id_pos']))
route['prior_stop_id'] = route['direction-stop_sequence'].apply(lambda x: stops_dict[x[0]][x[1]])
route = route.drop(columns='direction-stop_sequence')

# Timestamps

In [ ]:
# compute estimated number of seconds > 0 between stops
route['next_stop_est_sec'] = (route['next_stop_eta'] - route['timestamp']).dt.seconds

In [ ]:
# check for missing values in next_stop_est_sec and replace pursuant to strategies described below...
num_missing = route['next_stop_est_sec'].isna().sum()
if num_missing > 0:
  print(f'{num_missing} missing values in next_stop_est_sec... attempting to replace with median unique_trip_id value')
  uuids = list(route['unique_trip_id'])
  segment_times = list(route['next_stop_est_sec'])
  for (i, (uuid, segment_time)) in enumerate(zip(uuids, segment_times)):
    if pd.isna(segment_time):
      replacement_value = route[route['unique_trip_id'] == uuid]['next_stop_est_sec'].median()
      if not pd.isna(replacement_value):
        segment_times[i] = replacement_value
        num_missing -= 1
  route['next_stop_est_sec'] = segment_times
if num_missing > 0:
  print(f'{num_missing} missing values in next_stop_est_sec... attempting to replace with median trip_id value')
  trips = list(route['trip_id'])
  for (i, (trip, segment_time)) in enumerate(zip(trips, segment_times)):
    if pd.isna(segment_time):
      replacement_value = route[route['trip_id'] == trip]['next_stop_est_sec'].median()
      if not pd.isna(replacement_value):
        segment_times[i] = replacement_value
        num_missing -= 1
  route['next_stop_est_sec'] = segment_times    
if num_missing > 0:
  print(f'{num_missing} missing values in next_stop_est_sec... attempting to replace with median next_stop_id value')
  segments = list(route['next_stop_id'])
  for (i, (segment, segment_time)) in enumerate(zip(segments, segment_times)):
    if pd.isna(segment_time):
      replacement_value = route[route['next_stop_id'] == segment]['next_stop_est_sec'].median()
      if not pd.isna(replacement_value):
        segment_times[i] = replacement_value
        num_missing -= 1
  route['next_stop_est_sec'] = segment_times
if num_missing > 0:
  print(f'{num_missing} missing values in next_stop_est_sec... replacing with global average')
  replacement_value = route['next_stop_est_sec'].median()
  for (i, segment_time) in enumerate(segment_times):
    if pd.isna(segment_time):
      segment_times[i] = replacement_value
      num_missing -= 1
  segment_times = list(route['next_stop_est_sec'])
assert num_missing == 0
print(f'\nsuccessfully replaced all missing values in next_stop_est_sec!')

5070 missing values in next_stop_est_sec... attempting to replace with median unique_trip_id value
81 missing values in next_stop_est_sec... attempting to replace with median trip_id value
7 missing values in next_stop_est_sec... attempting to replace with median next_stop_id value

successfully replaced all missing values in next_stop_est_sec!


# Drop Columns and Save

In [ ]:
# drop unnecessary columns
keep = ['route', 'direction', 'trip_id', 'service_date', 'vehicle_id', 'timestamp', 'lat', 'lon', 'bearing', 'prior_stop_id', 'next_stop_id', 'next_stop_id_pos', 'next_stop_d_along_route', 'next_stop_est_sec', 'passenger_count']
route = route[keep]

In [ ]:
# save df to drive as csv
route.to_csv(save_segments, index=False)